In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# **GRUPO 5:**

* William Araujo / DNI: 44526862

* Angel Ramirez / DNI: 76594894

* Martina Guillermo / DNI: 47485009

* Pierre Padilla / DNI: 70254063

* Leonela Tacure / DNI: 4569538

**Fuente de datos:**

https://www.kaggle.com/alexandmax/covid-8-peru

https://www.kaggle.com/ppadillah/pruebasmoleculares-covid19-ins-peru

**Archivos:**

//positivos_covid.csv

//pm_19feb_2021.csv

//pm_ener_2021.csv

In [ ]:
!pip install pyspark

In [ ]:
from pyspark import SparkContext 
sc = SparkContext.getOrCreate()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)

# **DATASET 1 : DATA POSITIVO POR COVID-19 (positivos_covid.csv)**

**1. CARGAR DATOS AL KAGGLE**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)
POSITIVOS_DF =spark.read.load( "../input/covid-8-peru/positivos_covid.csv", format="csv", sep=";", header="true",inferSchema="true")
POSITIVOS_DF.show(10)

**2. CONTABILIZAR CUANTAS PERSONAS DIERON POSITIVO A COVID HASTA LA FECHA**


In [ ]:
NÚMERO_DE_POSITIVOS=POSITIVOS_DF.count()
print("El número de personas que dieron positivo hasta la fecha de corte 29/05/2021 es: \n",NÚMERO_DE_POSITIVOS)

In [ ]:
POSITIVOS_DF.createOrReplaceTempView( "users" )
spark.sql( "select count(*) from users" ).show() 

**3. CUÁNTOS CASOS POSITIVOS DE PERSONAS DE GÉNERO FEMENINAS Y MASCULINAS**

In [ ]:
POSITIVOS_SEX=POSITIVOS_DF.groupby("SEXO").count()
POSITIVOS_SEX.show()

In [ ]:
POSITIVOS_DF.createOrReplaceTempView( "users" )
spark.sql( "select case when SEXO is null then 'INDETERMINADO'\
 when SEXO == 'FEMENINO' then 'MUJER' \
 when SEXO == 'MASCULINO' then 'VARÓN' end GENERO_REGISTRADO, \
 count(*) from users group by GENERO_REGISTRADO order by 1" ).show()

**4. NÚMERO DE CASOS POSITIVOS POR DEPARTAMENTO**

In [ ]:
POSITIVOS_DEPARTAMENTO=POSITIVOS_DF.groupby("DEPARTAMENTO").count()
POSITIVOS_DEPARTAMENTO.show(26)

**5. TIPO DE PRUEBAS EMPLEADAS EN LA DETECCIÓN Y EL NÚMERO TOTAL DE CADA UNA.**

In [ ]:
TIPOS_PRUEBA1=POSITIVOS_DF.groupby("METODODX").count()
TIPOS_PRUEBA1.show()

**6. EL NÚMERO DE CASOS POR EDAD**

In [ ]:
TIPOS_PRUEBA2=POSITIVOS_DF.groupby("EDAD").count()
TIPOS_PRUEBA2.show()

In [ ]:
POSITIVOS_DF.createOrReplaceTempView( "users" )
RANGO=spark.sql( "select case when EDAD is null then 'Sin_registro'\
 when EDAD < 0 then 'Registro_negativo' \
 when EDAD between 0 and 24 then '0-25' \
 when EDAD between 25 and 39 then '25-40' \
 when EDAD >= 40 then '40+' end RANGO_EDAD, \
 count(*) from users group by RANGO_EDAD order by 1" )
RANGO.show()

**7. OPCIONAL: GRAFIQUE LOS RESULTADOS USANDO LAS BIBLIOTECAS ESTUDIADAS**

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark.sql as sparksql

In [ ]:
# Casos positivos de personas de género femeninas y masculinas

#POSITIVOS_SEX = POSITIVOS_DF.groupby("SEXO").count()
POSITIVOS_SEX.show()

x1 = POSITIVOS_SEX.toPandas()["SEXO"].values.tolist()
y1 = POSITIVOS_SEX.toPandas()["count"].values.tolist()

plt.bar(x1,y1)
plt.show


In [ ]:
# Número de casos positivos por departamento

x2 = POSITIVOS_DEPARTAMENTO.toPandas()["DEPARTAMENTO"].values.tolist()
y2 = POSITIVOS_DEPARTAMENTO.toPandas()["count"].values.tolist()

plt.bar(x2,y2)
plt.xticks(x2, rotation='vertical')
plt.show

In [ ]:
# Tipo de pruebas empleadas en la detección y el número total de cada una

x3 = TIPOS_PRUEBA1.toPandas()["METODODX"].values.tolist()
y3 = TIPOS_PRUEBA1.toPandas()["count"].values.tolist()

plt.bar(x3,y3)
plt.show

In [ ]:
# Número de casos por edad

x4 = RANGO.toPandas()["RANGO_EDAD"].values.tolist()
y4 = RANGO.toPandas()["count(1)"].values.tolist()

plt.bar(x4,y4)
plt.xticks(x4, rotation='vertical')
plt.show


# **DATASET 2: Pruebas moleculares para COVID-19 - 2021-**

# Fechas: Febrero y Marzo 2021

In [ ]:
import math
import numpy as np 
import pandas as pd  
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, isnull, asc, desc, mean

'''Create a spark session'''
spark = SparkSession.builder.master("local").appName("DataWrangling").getOrCreate()
'''Set this configuration to get output similar to pandas'''
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

**1. CARGAR DATOS AL KAGGLE**

In [ ]:
df_febrero = spark.read.csv('../input/pruebasmoleculares-covid19-ins-peru/pm_19feb_2021.csv',header=True)
df_febrero.limit(5)

In [ ]:
df_marzo = spark.read.csv('../input/covid-8-peru/pm12Marzo2021.csv',header=True, sep='|')
df_marzo.limit(5)

**2. NÚMERO TOTAL DE PRUEBAS POR MES**

In [ ]:
df_febrero.describe().show()

In [ ]:
df_febrero.agg({'FECHATOMAMUESTRA':'max'}).show()

In [ ]:
from pyspark.sql import functions as F
df_febrero.select(F.date_format('FECHATOMAMUESTRA','MM').alias('MES')).groupby('MES').count().sort('MES').show()

In [ ]:
df_marzo.select(F.date_format('FECHATOMAMUESTRA','MM').alias('MES')).groupby('MES').count().sort('MES').show()

**3. NÚMERO TOTAL DE PRUEBAS POR GÉNERO POR MES**

In [ ]:
febrero_sex = df_febrero.select(F.date_format('FECHATOMAMUESTRA','MM').alias('MES'), F.col('sexopaciente')).groupby('MES').pivot("sexopaciente").count().sort('MES')
febrero_sex.show()

In [ ]:
df_marzo.select(F.date_format('FECHATOMAMUESTRA','MM').alias('MES'), F.col('sexopaciente')).groupby('MES').pivot("sexopaciente").count().sort('MES').show()

**4. NÚMERO TOTAL DE PRUEBAS POR DEPARTAMENTO POR MES**

In [ ]:
df_febrero.select(F.date_format('FECHATOMAMUESTRA','MM').alias('MES'), F.col('DepOrigen')).groupby('DepOrigen').pivot("MES").count().sort('DepOrigen').show()

In [ ]:
df_marzo.select(F.date_format('FECHATOMAMUESTRA','MM').alias('MES'), F.col('DepOrigen')).groupby('DepOrigen').pivot("MES").count().sort('DepOrigen').show()

**5 y 6. NÚMERO TOTAL DE PRUEBAS POSITIVAS Y NEGATIVAS POR MES**

In [ ]:
df_febrero.select(F.date_format('FECHATOMAMUESTRA','MM').alias('MES'), F.col('RESULTADO')).groupby('MES').pivot("RESULTADO").count().sort('MES').show()

##### Hay datos distorcionados en la columna Resultados para el mes de Marzo por lo que se tiene que reemplazar los datos

In [ ]:
df_marzo.select('RESULTADO').distinct().show()

In [ ]:
df_marzo.select(F.col('RESULTADO')).groupby('RESULTADO').count()

In [ ]:
df_marzo = df_marzo.na.fill({'RESULTADO': 'INVALIDO'})

In [ ]:
df_marzo.select('RESULTADO').distinct().rdd.map(lambda r: r[0]).collect()

In [ ]:
df_marzo.select(F.date_format('FECHATOMAMUESTRA','MM').alias('MES'), F.col('RESULTADO')).groupby('MES').pivot("RESULTADO").count().sort('MES')

In [ ]:
df_marzo.select(F.date_format('FECHATOMAMUESTRA','MM').alias('MES'), F.col('RESULTADO')).groupby('RESULTADO').pivot("MES").count().sort('RESULTADO').show()

**7. GRÁFICOS**

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark.sql as sparksql

In [ ]:
febrero_sex = df_febrero.groupBy('sexopaciente').count()
febrero_sex.show()

In [ ]:
#Número total de pruebas por género por mes 

## MES: FEBRERO

a1 = febrero_sex.toPandas()["sexopaciente"].values.tolist()
b1 = febrero_sex.toPandas()["count"].values.tolist()

plt.bar(a1,b1)
plt.show

In [ ]:
marzo_sex = df_marzo.groupBy('sexopaciente').count()
marzo_sex.show()

In [ ]:
df_dep_febrero = df_febrero.groupBy('DepOrigen').count()
df_dep_febrero.show(30)

In [ ]:
# Número de pruebas por departamento por mes

##FEBRERO

a3 = df_dep_febrero.toPandas()["DepOrigen"].values.tolist()
b3 = df_dep_febrero.toPandas()["count"].values.tolist()

plt.bar(a3,b3)
plt.xticks(a3, rotation='vertical')
plt.show

In [ ]:
result_febrero = df_febrero.groupBy('RESULTADO').count()
result_febrero.show()

In [ ]:
#Número total de pruebas por resultado por mes 

## MES: FEBRERO

a4 = result_febrero.toPandas()["RESULTADO"].values.tolist()
b4 = result_febrero.toPandas()["count"].values.tolist()

plt.bar(a4,b4)
plt.show

In [ ]:
result_marzo = df_marzo.groupBy('RESULTADO').count()
result_marzo.show()

In [ ]:
#Número total de pruebas por resultado por mes 

## MES: MARZO

a5 = result_marzo.toPandas()["RESULTADO"].values.tolist()
b5 = result_marzo.toPandas()["count"].values.tolist()

plt.bar(a5,b5)
plt.show